In [20]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import random
import pickle
import time
import datetime
from collections import Counter
pd.set_option('display.max_rows',     1000)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',  None)

In [21]:
# Create the database connection here: 

db_connection_str = ...
connection = create_engine(db_connection_str)

___
# **Cohort Selection: Non-srugical adults**

In [22]:
all_hamds_query = 'SELECT DISTINCT HADM_ID FROM ADMISSIONS'
all_hamd_IDs_df = pd.read_sql(all_hamds_query, con=connection)
print('Count of all HADM_IDs: {:,}'.format(len(all_hamd_IDs_df)))

Count of all HADM_IDs: 58,976


### All (MetaVision/metavision):

In [23]:
dbase_query = 'SELECT DISTINCT HADM_ID, DBSOURCE from ICUSTAYS'
dbase_df    = pd.read_sql(dbase_query, con=connection)
all_carevue_hadmIDs    = dbase_df[dbase_df.DBSOURCE=='carevue'].HADM_ID.tolist()
all_metavision_hadmIDs = dbase_df[dbase_df.DBSOURCE=='metavision'].HADM_ID.tolist()
print('Count of all CareVue    HADM_IDs: {:,}'.format(len(all_carevue_hadmIDs)))
print('Count of all MetaVision HADM_IDs: {:,}'.format(len(all_metavision_hadmIDs)))


Count of all CareVue    HADM_IDs: 35,632
Count of all MetaVision HADM_IDs: 22,046


### Adult HADM_IDs:

In [24]:
adult_query = '''
SELECT A.HADM_ID, DATEDIFF(A.ADMITTIME,DOB)/365 AS AGE_at_admission
FROM ADMISSIONS A JOIN PATIENTS P ON A.SUBJECT_ID=P.SUBJECT_ID
HAVING AGE_at_admission BETWEEN 18 AND 100
'''

all_adults_df      = pd.read_sql(adult_query, con=connection)
all_adult_hadm_ids = all_adults_df.HADM_ID.unique().tolist()
print('Count of adult HADM_IDs:            {:,}'.format(len(all_adult_hadm_ids)))

Count of adult HADM_IDs:            48,150


In [25]:
adult_carevue_hadm_ids          = list(set(all_carevue_hadmIDs)   &set(all_adult_hadm_ids))
adult_metavision_adult_hadm_ids = list(set(all_metavision_hadmIDs)&set(all_adult_hadm_ids))

print('Count of adult CareVue    HADM_IDs: {:,}'.format(len(adult_carevue_hadm_ids)))
print('Count of adult MetaVision HADM_IDs: {:,}'.format(len(adult_metavision_adult_hadm_ids)))


Count of adult CareVue    HADM_IDs: 26,197
Count of adult MetaVision HADM_IDs: 20,811


### Non-surgical:

In [26]:
services_query = 'SELECT HADM_ID,CURR_SERVICE FROM SERVICES WHERE HADM_ID IN {}'.format(tuple(all_adult_hadm_ids))
services_df    = pd.read_sql_query(services_query, connection)
print('Count of adult HADM_IDs with services recorded: {:,}'.format(len(services_df.HADM_ID.unique())))

Count of adult HADM_IDs with services recorded: 48,117


In [27]:
services_mapping = {'CMED' : True,  'CSURG':  False,  'DENT' : False, 'ENT' : False,
                    'GU' :   False, 'GYN' :   False,  'MED' :  True,  'NB' : False,
                    'NBB' :  False, 'NMED' :  True,   'NSURG': False, 'OBS' : False,
                    'ORTHO': False, 'OMED' :  True,   'PSURG': False, 'PSYCH' : False,
                    'SURG' : False, 'TRAUM' : False,  'TSURG': False, 'VSURG' : False}

In [28]:
services_df['Include'] = services_df.apply(lambda row: services_mapping[row['CURR_SERVICE']], axis = 1)
df = services_df.groupby('HADM_ID')['Include'].all().reset_index()
adult_nonSurg_hadm_IDs = df[df['Include'] == True].HADM_ID.tolist()
print('Count of non-surgical adult hadm_IDs CareVue and Metavision combnied: {:,}'.format(len(adult_nonSurg_hadm_IDs)))


adult_nonSurg_carevue_hadm_IDs    = list(set(adult_carevue_hadm_ids)   & set(adult_nonSurg_hadm_IDs))
adult_nonSurg_metavision_hadm_IDs = list(set(adult_metavision_adult_hadm_ids)& set(adult_nonSurg_hadm_IDs))

print('Count of adult non-surgical CareVue    HADM_IDs: {:,}'.format(len(adult_nonSurg_carevue_hadm_IDs)))
print('Count of adult non-surgical MetaVision HADM_IDs: {:,}'.format(len(adult_nonSurg_metavision_hadm_IDs)))

Count of non-surgical adult hadm_IDs CareVue and Metavision combnied: 24,942
Count of adult non-surgical CareVue    HADM_IDs: 13,157
Count of adult non-surgical MetaVision HADM_IDs: 11,640


________________
# **Functions:**

In [15]:
def chartlab_vectorizer(hadm_ID, conn=connection):
    '''
    This functions vectorizes all relavant chart and lab items.
    '''
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    chart_item_dict = {444:'Mean_Airway_P', 224697: 'Mean_Airway_P',
                       535:'Peak',          224695: 'Peak',        
                       505:'PEEP',          506:    'PEEP',   220339: 'PEEP',
                       543:'Plateau_P',     224696:'Plateau_P',    
                       682:'TV_Obsed',      224685:'TV_Obsed',     
                       683:'TV_Set',        224684:'TV_Set',       
                       684:'TV_Spont',      224686:'TV_Spont',     
                       615:'RR_Total',      224690:'RR_Total',     
                       618:'RR',            220210:'RR',           
                       619:'RR_Set',        224688:'RR_Set',       
                       614:'RR_Spont',      224689:'RR_Spont',     
                       722:'Vent_Type',     223848:'Vent_Type',    
                       720:'Vent_Mode',     223849:'Vent_Mode',    
                       646:'SpO2',          220277:'SpO2',         
                       834:'SaO2_chart',    220227:'SaO2_chart',   
                       190:'FiO2_decimal_chart',                   
                       3420:'FiO2_percent_chart',   223835:'FiO2_percent_chart',   
                       779:'PaO2_chart',    220224:'PaO2_chart',   
                       778:'PaCO2_chart',   220235:'PaCO2_chart',  
                       3810:'TCO2_chart',   225698:'TCO2_chart',   
                       780:'PH_chart',      223830:'PH_chart',     
                       470:'O2_Flow',       223834:'O2_Flow',      
                       467:'O2_Device',     226732:'O2_Device',    
                       468:'O2_Device2',                           
                       471:'O2_Flow2',
                      }
    text_items = [722,223848,720,223849,467,226732,468] #These are text items for which we should get VALUE not VALUENUM
    chart_query = '''
        SELECT  HADM_ID,SUBJECT_ID,ITEMID,CHARTTIME, VALUE, VALUENUM
        FROM CHARTEVENTS 
        WHERE HADM_ID = {} 
        AND ITEMID in {}
        AND (ERROR IS Null OR ERROR = 0) AND VALUE IS NOT NULL
        order by CHARTTIME
    '''.format(hadm_ID, tuple(chart_item_dict.keys()))
    
    chart_df = pd.read_sql_query(chart_query,conn)
    vectorized_chart_df = pd.DataFrame(columns=['HADM_ID_chart', 'SUBJECT_ID_chart']+sorted([item for item in list(set(chart_item_dict.values()))]))
    dtms   = chart_df.CHARTTIME.unique()
    vectorized_chart_df['CHARTTIME']        = dtms
    vectorized_chart_df['HADM_ID_chart']    = hadm_ID
    
    if len(chart_df)>0: 
        vectorized_chart_df['SUBJECT_ID_chart'] = chart_df['SUBJECT_ID'].values[0] 
    else: 
        vectorized_chart_df['SUBJECT_ID_chart'] = np.NaN
    vectorized_chart_df = vectorized_chart_df.set_index(['CHARTTIME'])
    
    for index,row in chart_df.iterrows():
        if row['ITEMID'] in text_items:
            vectorized_chart_df.loc[row['CHARTTIME'],chart_item_dict[row['ITEMID']]] = row['VALUE']
        else:
            vectorized_chart_df.loc[row['CHARTTIME'],chart_item_dict[row['ITEMID']]] = row['VALUENUM']
        
    vectorized_chart_df = vectorized_chart_df.dropna(how = 'all').reset_index()
    vectorized_chart_df['FiO2_chart'] = vectorized_chart_df['FiO2_percent_chart'].fillna(vectorized_chart_df['FiO2_decimal_chart']*100)
    vectorized_chart_df.drop(['FiO2_decimal_chart','FiO2_percent_chart'],axis = 1, inplace=True)
    
    #______________________________________________________________________________________________________________________
    #______________________________________________________________________________________________________________________
    #______________________________________________________________________________________________________________________
                                                                                                                   
    lab_item_dict = {50817:'SaO2_lab',
                     50816:'FiO2_lab',
                     50821:'PaO2_lab',
                     50818:'PaCO2_lab',
                     50804:'TCO2_lab',
                     50820: 'PH_lab'
                      }
    lab_query = '''
    SELECT  HADM_ID,SUBJECT_ID,ITEMID,CHARTTIME, VALUE, VALUENUM
    FROM LABEVENTS 
    WHERE HADM_ID = {} 
    AND ITEMID in {}
    AND VALUE IS NOT NULL
    order by CHARTTIME
    '''.format(hadm_ID, tuple(lab_item_dict.keys()))
    lab_df = pd.read_sql_query(lab_query,conn)
    vectorized_lab_df = pd.DataFrame(columns=['HADM_ID_lab', 'SUBJECT_ID_lab'] + [item for item in list(set(lab_item_dict.values()))])
    dtms   = lab_df.CHARTTIME.unique() 
    vectorized_lab_df['CHARTTIME']   = dtms
    vectorized_lab_df['HADM_ID_lab'] = hadm_ID
    
    if len(lab_df)>0: 
        vectorized_lab_df['SUBJECT_ID_lab'] = lab_df['SUBJECT_ID'].values[0] 
    else: 
        vectorized_lab_df['SUBJECT_ID_lab'] = np.NaN
        
        
    vectorized_lab_df = vectorized_lab_df.set_index(['CHARTTIME'])
    for index,row in lab_df.iterrows():
        vectorized_lab_df.loc[row['CHARTTIME'],lab_item_dict[row['ITEMID']]] = row['VALUENUM']
        
    vectorized_lab_df = vectorized_lab_df.dropna(how = 'all').reset_index()
    
    combined_vectorized_df = pd.merge(vectorized_chart_df,vectorized_lab_df, how ='outer', on = 'CHARTTIME' )
    combined_vectorized_df.sort_values(by = 'CHARTTIME', inplace = True)
   
    
    # Giving priority to Lab data: 
    for measure in ['PaO2','FiO2', 'PaCO2', 'PH', 'TCO2', 'SaO2', 'HADM_ID', 'SUBJECT_ID']:
        combined_vectorized_df[measure] = combined_vectorized_df[measure +'_lab'].fillna(combined_vectorized_df[measure +'_chart'])
        combined_vectorized_df.drop([measure +'_lab',measure +'_chart'],axis = 1, inplace=True)
    

    # When O2_device is None flow is zero:
    combined_vectorized_df.loc[combined_vectorized_df['O2_Device']=='None','O2_Flow']   = 0
    combined_vectorized_df.loc[combined_vectorized_df['O2_Device2']=='None','O2_Flow2'] = 0

    combined_vectorized_df = combined_vectorized_df.reindex(['SUBJECT_ID','HADM_ID'] + sorted(combined_vectorized_df.columns.drop(['HADM_ID','SUBJECT_ID'])), axis=1)
    return combined_vectorized_df

In [16]:
def T_intub_finder(hadm_ID,ChartLab_vectorized_df, conn=connection):
    '''
    This function finds the time of intubation following the logic explained in the paper. 
    The output is the list of time of intubation based on each parameter:
    output: [first_vent_parameter_dtm,first_O2_delivery_method_dtm, procedure_dtm, paralytics_dtm]
    '''
    import warnings
    import datetime

    warnings.filterwarnings("ignore", 'This pattern has match groups')
    
    ChartLab_vectorized_df = chartlab_vectorizer(hadm_ID, conn)
    ##______________________________________________________________________________________________________________
    # First vent_parameter dtm
    vent_parameter_df = \
    ChartLab_vectorized_df[(\
                 (pd.notnull(ChartLab_vectorized_df.Mean_Airway_P))|\
                 (pd.notnull(ChartLab_vectorized_df.Peak))|\
                 (pd.notnull(ChartLab_vectorized_df.PEEP))|\
                 (pd.notnull(ChartLab_vectorized_df.Plateau_P))|\
                 (pd.notnull(ChartLab_vectorized_df.TV_Obsed))|\
                 (pd.notnull(ChartLab_vectorized_df.TV_Set))|\
                 (pd.notnull(ChartLab_vectorized_df.TV_Spont))\
                )\
                & (~ChartLab_vectorized_df.Vent_Type.astype(str).str.contains(r'(NIV|VAPS|BiPAP|CPAP)', regex=True,case=False))\
                & (~ChartLab_vectorized_df.Vent_Mode.astype(str).str.contains(r'(NIV|VAPS|BiPAP|CPAP)', regex=True,case=False))\
               ]
    first_vent_parameter_dtm = vent_parameter_df.CHARTTIME.min()
    ##______________________________________________________________________________________________________________
    ##______________________________________________________________________________________________________________
    #  O2_delivery method:
    O2_delivery_method_df = \
    ChartLab_vectorized_df[(ChartLab_vectorized_df.O2_Device == 'Ventilator')|
                         (ChartLab_vectorized_df.O2_Device == 'Endotracheal tube')]
    first_O2_delivery_method_dtm = O2_delivery_method_df.CHARTTIME.min()
    
    ##______________________________________________________________________________________________________________
    ##______________________________________________________________________________________________________________
    #  Intubation procdure in the PROCEDUREEVENTS_MV, Only applies to Metavision patients. (Item 224385	Intubation	4,514 patients)
    procedure_query = '''
    SELECT ENDTIME
    FROM PROCEDUREEVENTS_MV
    WHERE HADM_ID = {} AND ITEMID = 224385
    AND STATUSDESCRIPTION LIKE 'FinishedRunning'
    '''.format(hadm_ID)
    procedure_df = pd.read_sql(procedure_query, con=connection)
    procedure_dtm = procedure_df.ENDTIME.min()
    ##______________________________________________________________________________________________________________
    ##______________________________________________________________________________________________________________
    #  Paralytic drugs. We need to check if the patient is MV or CV.
    
    dbase = np.NaN
    dbase_query = '''SELECT DBSOURCE from ICUSTAYS WHERE HADM_ID = {}'''.format(hadm_ID)
    dbase_df = pd.read_sql(dbase_query, con=connection)
    if len(dbase_df)>0:
        dbase = dbase_df.DBSOURCE.tolist()[0]        
    # DBSOURCE is 'metavision', 'carevue', or 'both'. There are only 150 'both's. I will just query carevue for those cases. Risk is mitigated by other T_0 factors.

    if dbase == 'metavision':
        # ITEMID	Label	CNT
        # 222168	Propofol	178,819             ++++++
        # 221744	Fentanyl	86,340              ++++++
        # 225942	Fentanyl (Concentrate)	45,866
        # 221555	Cisatracurium	9,334           ++++++
        # 222062	Vecuronium	664                 ++++++
        paralytics_query = '''
        SELECT STARTTIME
        FROM INPUTEVENTS_MV
        WHERE HADM_ID = {} 
        AND ( 
              (ITEMID in (222168,221744,225942) AND (ORDERCATEGORYNAME LIKE '01-Drips')) 
             OR 
              (ITEMID in (221555,222062))
              )
        '''.format(hadm_ID)
        paralytics_df  = pd.read_sql(paralytics_query, con=connection)
        paralytics_dtm = paralytics_df.STARTTIME.min()
    else:
        # 30131 	Propofol	924,614      ++++++
        # 30118 	Fentanyl	780,555      ++++++
        # 30114 	Cisatracurium	63,994   ++++++
        # 30308 	Fentanyl Drip	36,595
        # 30149 	Fentanyl (Conc)	35,526
        # 30150 	Fentanyl Base	14,849
        # 30138 	Vecuronium	5,160        ++++++
        paralytics_query = '''
        SELECT CHARTTIME
        FROM INPUTEVENTS_CV
        WHERE HADM_ID = {} 
        AND (
              (ITEMID in (30131,30118,30308,30149,30150) AND (ORIGINALROUTE LIKE 'IV Drip' OR ORIGINALROUTE LIKE 'Intravenous Infusion' OR ORIGINALROUTE LIKE 'Drip')) 
             OR
              (ITEMID in (30114,30138))
            )
        '''.format(hadm_ID)
        paralytics_df = pd.read_sql(paralytics_query, con=connection)
        paralytics_dtm = paralytics_df.CHARTTIME.min()

    T_intubs = [first_vent_parameter_dtm,first_O2_delivery_method_dtm, procedure_dtm, paralytics_dtm]

    
    return T_intubs

In [17]:
chartlab_df = chartlab_vectorizer(106266, conn=connection)
t_list = T_intub_finder(106266,chartlab_df, conn=connection)
t_list


[Timestamp('2114-11-29 22:00:00'), Timestamp('2114-11-29 22:00:00'), nan, nan]

_______________
# **Running the code for all patients in the cohort:**

In [14]:
not_intubated_hadmIDs      = []
T_intubs_dict_for_analysis = dict()
start_time                 = time.time()
counter                    = 0

for hadmID in adult_nonSurg_hadm_IDs:
    if hadmID in T_intubs_dict_for_analysis.keys():
        pass
    else:
        chartlab_df = chartlab_vectorizer(hadmID, conn=connection)
        result      = T_intub_finder(hadmID,chartlab_df, conn=connection)
        if any([pd.notnull(T) for T in result]):
            T_intubs_dict_for_analysis.update({hadmID:result})
#             print(result)
        else:
            not_intubated_hadmIDs.append(hadmID)
        counter+=1
        if counter%1000 == 0:
            print('-----{} of {} done in {:.2f} minutes!'.format(counter,len(all_adult_hadm_ids),(time.time() - start_time)/60))
            with open('T_intubs_dict_for_analysis','wb') as file:
                pickle.dump(T_intubs_dict_for_analysis, file)
            with open('not_intubated_hadmIDs','wb') as file:
                pickle.dump(not_intubated_hadmIDs, file)

with open('T_intubs_dict_for_analysis','wb') as file:
    pickle.dump(T_intubs_dict_for_analysis, file) 
with open('not_intubated_hadmIDs','wb') as file:
    pickle.dump(not_intubated_hadmIDs, file) 

-----1000 of 48150 done in 12.65 minutes!
-----2000 of 48150 done in 24.79 minutes!
-----3000 of 48150 done in 37.43 minutes!
-----4000 of 48150 done in 51.32 minutes!
-----5000 of 48150 done in 63.26 minutes!
-----6000 of 48150 done in 74.20 minutes!
-----7000 of 48150 done in 93.58 minutes!
-----8000 of 48150 done in 106.92 minutes!
-----9000 of 48150 done in 121.58 minutes!
-----10000 of 48150 done in 159.12 minutes!
-----11000 of 48150 done in 171.61 minutes!
-----12000 of 48150 done in 185.46 minutes!
-----13000 of 48150 done in 198.01 minutes!
-----14000 of 48150 done in 208.91 minutes!
-----15000 of 48150 done in 221.77 minutes!
-----16000 of 48150 done in 235.23 minutes!
-----17000 of 48150 done in 247.59 minutes!
-----18000 of 48150 done in 260.82 minutes!
-----19000 of 48150 done in 272.26 minutes!
-----20000 of 48150 done in 284.88 minutes!
-----21000 of 48150 done in 299.05 minutes!
-----22000 of 48150 done in 315.37 minutes!
-----23000 of 48150 done in 326.97 minutes!
----

___________________
# **Analysis of $\Delta$T:**

In [18]:
with open('T_intubs_dict_for_analysis','rb') as file:
    T_intubs_dict_for_analysis = pickle.load(file) 
with open('not_intubated_hadmIDs','rb') as file:
    not_intubated_hadmIDs = pickle.load(file) 

## **Carevue:**

In [29]:
adult_nonSurg_intubated_carevue_hadm_IDs    = list(set(adult_nonSurg_carevue_hadm_IDs)   - set(not_intubated_hadmIDs))
print('Count of adult non-surgical CareVue HADM_IDs: {:,}'.format(len(adult_nonSurg_carevue_hadm_IDs)))
print('Count of intubated carevue HADM_IDs:          {:,}'.format(len(adult_nonSurg_intubated_carevue_hadm_IDs)))
print('% of intubated carevue HADM_IDs:              {:.1f}% '.format(100*len(adult_nonSurg_intubated_carevue_hadm_IDs)/len(adult_nonSurg_carevue_hadm_IDs)))

Count of adult non-surgical CareVue HADM_IDs: 13,157
Count of intubated carevue HADM_IDs:          4,657
% of intubated carevue HADM_IDs:              35.4% 


In [30]:
factors = ['vent_parameter', 'O2_delivery_method','procedure', 'paralytics']
determining_factor_list_carevue    = []
delta_Ts_carevue                   = []
no_vent_parameter_recorded_carevue = []
for hadmID in adult_nonSurg_intubated_carevue_hadm_IDs:
#     print(T_intubs_dict_for_analysis[hadmID])
    
    # Getting intubated hadm_IDs without a vent parameter:
    if pd.isnull(T_intubs_dict_for_analysis[hadmID][0]):
        no_vent_parameter_recorded_carevue.append(hadmID)
    
    # Getting delta T's:
    T_intub = min([T for T in T_intubs_dict_for_analysis[hadmID] if (isinstance(T, datetime.datetime) & pd.notnull(T))])
    determining_indices = [index for index, value in enumerate(T_intubs_dict_for_analysis[hadmID]) if value == T_intub]
    for determining_index in determining_indices:
        determining_factor_list_carevue.append(factors[determining_index])
#     print(hadmID, T_intubs_dict_for_analysis[hadmID][0])
    if pd.notnull(T_intubs_dict_for_analysis[hadmID][0]):
        delta_T_Tintub_and_vent_parameter = (T_intubs_dict_for_analysis[hadmID][0] - T_intub).total_seconds()/3600
        delta_Ts_carevue.append(delta_T_Tintub_and_vent_parameter)

In [31]:
print('Count of intubated carevue HADM_IDs with vent parameter missing:  {:,}'.format(len(no_vent_parameter_recorded_carevue)))
print('% of intubated carevue HADM_ID with vent parameter missings:      {:.1f}%'.format(100*len(no_vent_parameter_recorded_carevue)/len(adult_nonSurg_intubated_carevue_hadm_IDs)))

Count of intubated carevue HADM_IDs with vent parameter missing:  54
% of intubated carevue HADM_ID with vent parameter missings:      1.2%


In [19]:
counts_dict = dict(Counter(determining_factor_list_carevue))
for factor in factors:
    if factor in counts_dict.keys():
        print('{}: {:,} ({:.1f}%)'.format(factor,counts_dict[factor],counts_dict[factor]/len(determining_factor_list_carevue)*100))

vent_parameter: 4,250 (85.6%)
procedure: 1 (0.0%)
paralytics: 714 (14.4%)


In [20]:
print('Non-zero Delta_Ts (T_intub - T_vent_parameter):\n')

Non_zero_delta_Ts_carevue = [t for t in delta_Ts_carevue if t!=0]
print('Count:  {:,} ({:.1f}%)'.format(len(Non_zero_delta_Ts_carevue),100*len(Non_zero_delta_Ts_carevue)/len(delta_Ts_carevue)))

max_t = max(Non_zero_delta_Ts_carevue)
min_t = min(Non_zero_delta_Ts_carevue)
        
print('\nRange:  {:.2f} ({:.2f},{:.2f})'.format(max_t-min_t, min_t, max_t))
print('\nMean:   {:.1f} ({:.1f})'.format(np.mean(Non_zero_delta_Ts_carevue),np.std(Non_zero_delta_Ts_carevue,ddof=1)))
median = np.median(Non_zero_delta_Ts_carevue)
q1 =     np.percentile(Non_zero_delta_Ts_carevue,25)
q3 =     np.percentile(Non_zero_delta_Ts_carevue,75)
print('\nMedian: {:.2f} ({:.2f},{:.2f})'.format(median,q1,q3))
print('\n{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_carevue if t>1]),100*(len([t for t in Non_zero_delta_Ts_carevue if t>1])/len(adult_nonSurg_intubated_carevue_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_carevue if t>2]),100*(len([t for t in Non_zero_delta_Ts_carevue if t>2])/len(adult_nonSurg_intubated_carevue_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_carevue if t>4]),100*(len([t for t in Non_zero_delta_Ts_carevue if t>4])/len(adult_nonSurg_intubated_carevue_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_carevue if t>8]),100*(len([t for t in Non_zero_delta_Ts_carevue if t>8])/len(adult_nonSurg_intubated_carevue_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_carevue if t>12]),100*(len([t for t in Non_zero_delta_Ts_carevue if t>12])/len(adult_nonSurg_intubated_carevue_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_carevue if t>24]),100*(len([t for t in Non_zero_delta_Ts_carevue if t>24])/len(adult_nonSurg_intubated_carevue_hadm_IDs))))

Non-zero Delta_Ts (T_intub - T_vent_parameter):

Count:  353 (7.7%)

Range:  572.98 (0.02,573.00)

Mean:   6.4 (34.8)

Median: 0.50 (0.25,1.25)

95 (2.0%)
67 (1.4%)
47 (1.0%)
40 (0.9%)
28 (0.6%)
17 (0.4%)


## **MetaVision**: 

In [32]:
adult_nonSurg_intubated_metavision_hadm_IDs = list(set(adult_nonSurg_metavision_hadm_IDs)- set(not_intubated_hadmIDs))
print('Count of adult non-surgical MetaVision HADM_IDs: {:,}'.format(len(adult_nonSurg_metavision_hadm_IDs)))
print('Count of intubated metavision HADM_IDs:          {:,}'.format(len(adult_nonSurg_intubated_metavision_hadm_IDs)))
print('% of intubated metavision HADM_IDs:              {:.1f}% '.format(100*len(adult_nonSurg_intubated_metavision_hadm_IDs)/len(adult_nonSurg_metavision_hadm_IDs)))

Count of adult non-surgical MetaVision HADM_IDs: 11,640
Count of intubated metavision HADM_IDs:          3,600
% of intubated metavision HADM_IDs:              30.9% 


In [33]:
factors = ['vent_parameter', 'O2_delivery_method','procedure', 'paralytics']
determining_factor_list_metavision    = []
delta_Ts_metavision                   = []
no_vent_parameter_recorded_metavision = []
for hadmID in adult_nonSurg_intubated_metavision_hadm_IDs:
#     print(T_intubs_dict_for_analysis[hadmID])
    
    # Getting intubated hadm_IDs without a vent parameter:
    if pd.isnull(T_intubs_dict_for_analysis[hadmID][0]):
        no_vent_parameter_recorded_metavision.append(hadmID)
    
    # Getting delta T's:
    T_intub = min([T for T in T_intubs_dict_for_analysis[hadmID] if (isinstance(T, datetime.datetime) & pd.notnull(T))])
    determining_indices = [index for index, value in enumerate(T_intubs_dict_for_analysis[hadmID]) if value == T_intub]
    for determining_index in determining_indices:
        determining_factor_list_metavision.append(factors[determining_index])
#     print(hadmID, T_intubs_dict_for_analysis[hadmID][0])
    if pd.notnull(T_intubs_dict_for_analysis[hadmID][0]):
        delta_T_Tintub_and_vent_parameter = (T_intubs_dict_for_analysis[hadmID][0] - T_intub).total_seconds()/3600
        delta_Ts_metavision.append(delta_T_Tintub_and_vent_parameter)

In [34]:
print('Count of intubated metavision HADM_IDs with vent parameter missing:  {:,}'.format(len(no_vent_parameter_recorded_metavision)))
print('% of intubated metavision HADM_ID with vent parameter missings:      {:.2f}%'.format(100*len(no_vent_parameter_recorded_metavision)/len(adult_nonSurg_intubated_metavision_hadm_IDs)))

Count of intubated metavision HADM_IDs with vent parameter missing:  109
% of intubated metavision HADM_ID with vent parameter missings:      3.03%


In [24]:
counts_dict = dict(Counter(determining_factor_list_metavision))
for factor in factors:
    if factor in counts_dict.keys():
        print('{}: {:,} ({:.1f}%)'.format(factor,counts_dict[factor],counts_dict[factor]/len(determining_factor_list_metavision)*100))

vent_parameter: 2,435 (50.2%)
O2_delivery_method: 1,281 (26.4%)
procedure: 410 (8.5%)
paralytics: 726 (15.0%)


In [25]:
print('Non-zero Delta_Ts (T_i - T_v):\n')

Non_zero_delta_Ts_metavision = [t for t in delta_Ts_metavision if t!=0]
print('Count:  {:,} ({:.1f}%)'.format(len(Non_zero_delta_Ts_metavision),100*len(Non_zero_delta_Ts_metavision)/len(delta_Ts_metavision)))

max_t = max(Non_zero_delta_Ts_metavision)
min_t = min(Non_zero_delta_Ts_metavision)
        
print('\nRange:  {:.2f} ({:.2f},{:.2f})'.format(max_t-min_t, min_t, max_t))
print('\nMean:   {:.1f} ({:.1f})'.format(np.mean(Non_zero_delta_Ts_metavision),np.std(Non_zero_delta_Ts_metavision,ddof=1)))
median = np.median(Non_zero_delta_Ts_metavision)
q1 =     np.percentile(Non_zero_delta_Ts_metavision,25)
q3 =     np.percentile(Non_zero_delta_Ts_metavision,75)
print('\nMedian: {:.2f} ({:.2f},{:.2f})'.format(median,q1,q3))
print('\n{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_metavision if t>1]),100*(len([t for t in Non_zero_delta_Ts_metavision if t>1])/len(adult_nonSurg_intubated_metavision_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_metavision if t>2]),100*(len([t for t in Non_zero_delta_Ts_metavision if t>2])/len(adult_nonSurg_intubated_metavision_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_metavision if t>4]),100*(len([t for t in Non_zero_delta_Ts_metavision if t>4])/len(adult_nonSurg_intubated_metavision_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_metavision if t>8]),100*(len([t for t in Non_zero_delta_Ts_metavision if t>8])/len(adult_nonSurg_intubated_metavision_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_metavision if t>12]),100*(len([t for t in Non_zero_delta_Ts_metavision if t>12])/len(adult_nonSurg_intubated_metavision_hadm_IDs))))
print('{} ({:.1f}%)'.format(len([t for t in Non_zero_delta_Ts_metavision if t>24]),100*(len([t for t in Non_zero_delta_Ts_metavision if t>24])/len(adult_nonSurg_intubated_metavision_hadm_IDs))))

Non-zero Delta_Ts (T_i - T_v):

Count:  1,056 (30.2%)

Range:  310.07 (0.02,310.08)

Mean:   2.5 (12.0)

Median: 0.48 (0.18,1.00)

253 (7.0%)
151 (4.2%)
105 (2.9%)
70 (1.9%)
49 (1.4%)
19 (0.5%)


____
# **Finding each pateint's full time series:**

In [39]:
def T_extub_finder(hadm_ID,ChartLab_vectorized_df, conn=connection):
    import warnings
    import datetime
    
    ChartLab_vectorized_df = chartlab_vectorizer(hadm_ID, conn)
    ##______________________________________________________________________________________________________________
    # Last vent_parameter dtm
    vent_parameter_df = ChartLab_vectorized_df[(
        (pd.notnull(ChartLab_vectorized_df.Mean_Airway_P))|
        (pd.notnull(ChartLab_vectorized_df.Peak))|
        (pd.notnull(ChartLab_vectorized_df.PEEP))|
        (pd.notnull(ChartLab_vectorized_df.Plateau_P))|
        (pd.notnull(ChartLab_vectorized_df.TV_Obsed))|
        (pd.notnull(ChartLab_vectorized_df.TV_Set))|
        (pd.notnull(ChartLab_vectorized_df.TV_Spont))|
        (ChartLab_vectorized_df.O2_Device == 'Ventilator'))
        & (~ChartLab_vectorized_df.Vent_Type.astype(str).str.contains(r'(NIV|VAPS|BiPAP|CPAP)', regex=True,case=False))
        & (~ChartLab_vectorized_df.Vent_Mode.astype(str).str.contains(r'(NIV|VAPS|BiPAP|CPAP)', regex=True,case=False))]
    last_vent_parameter_dtm = vent_parameter_df.CHARTTIME.max()
    ##______________________________________________________________________________________________________________
    ##______________________________________________________________________________________________________________
    #  Extubation procedure in the PROCEDUREEVENTS_MV, Only applies to Metavision patients.     
        # ITEMID	Label	CNT
        # 0	227194	Extubation	8328
        # 1	225468	Unplanned Extubation (patient-initiated)	327
        # 2	225477	Unplanned Extubation (non-patient initiated)	20
    
    procedure_query = '''
    SELECT ENDTIME
    FROM PROCEDUREEVENTS_MV
    WHERE HADM_ID = {} AND ITEMID in (227194, 225468, 225477)
    AND STATUSDESCRIPTION LIKE 'FinishedRunning'
    '''.format(hadm_ID)
    procedure_df = pd.read_sql(procedure_query, con=connection)
    procedure_dtm = procedure_df.ENDTIME.min()
    ##______________________________________________________________________________________________________________
    ##______________________________________________________________________________________________________________
    
    # First non-vent O2 delivery method after last vent parameter:
    NoneVent_O2_method_after_intub_df = ChartLab_vectorized_df[(ChartLab_vectorized_df.CHARTTIME>last_vent_parameter_dtm)& \
                                                             (pd.notnull(ChartLab_vectorized_df.O2_Device)) & \
                                                             (~ChartLab_vectorized_df.Vent_Mode.astype(str).str.contains(r'(ventilator)', regex=True,case=False))]
    first_NoneVent_method_after_intub_dtm = NoneVent_O2_method_after_intub_df.CHARTTIME.min()
    
    
    T_extubs = [procedure_dtm, first_NoneVent_method_after_intub_dtm]
    try:
        T_extub = min([T for T in T_extubs if ((isinstance(T, datetime.datetime)) & (pd.notnull(T)))])
    except ValueError:
        T_extub = np.NaN

    
    return T_extub

In [40]:
def add_event(ChartLab_vectorized_df, event_name, event_dtm):
    '''
    Sometimes things overwrite each other. For example, admission may happen at the same time as intubation. To avoid this, 
    I will not add the event to the existing dtm. I have done this with the following but later I changed it:
    '''

    if pd.notnull(event_dtm):
        ChartLab_vectorized_df = ChartLab_vectorized_df.append({'CHARTTIME':event_dtm,'Events': event_name}, ignore_index = True)
        return ChartLab_vectorized_df
    else:
        return ChartLab_vectorized_df

In [41]:
def fill_forward_interventions (ChartLab_vectorized_df, T_Intub, T_Extub,
                                off_vent_interventions = ['O2_Device', 'O2_Flow','O2_Device2', 'O2_Flow2','Vent_Mode','Vent_Type'],
                                on_vent_interventions  = ['FiO2', 'PEEP', 'RR_Set','TV_Set','Vent_Mode','Vent_Type']):    
    
    # If intubated:
    if pd.notnull(T_Intub):
        # ffil off_vent_interventions before intubation:
#         display(ChartLab_vectorized_df)
        ChartLab_vectorized_df.loc[ChartLab_vectorized_df.CHARTTIME<T_Intub,off_vent_interventions] =\
        ChartLab_vectorized_df.loc[ChartLab_vectorized_df.CHARTTIME<T_Intub,off_vent_interventions].ffill()
        
        # If extubated:
        if pd.notnull(T_Extub):
            # ffil on_vent_interventions between intubation and extubation:
            ChartLab_vectorized_df.loc[(ChartLab_vectorized_df.CHARTTIME>=T_Intub)&(ChartLab_vectorized_df.CHARTTIME<T_Extub),on_vent_interventions] =\
            ChartLab_vectorized_df.loc[(ChartLab_vectorized_df.CHARTTIME>=T_Intub)&(ChartLab_vectorized_df.CHARTTIME<T_Extub),on_vent_interventions].ffill()
            # ffil off_vent_interventions after extubation:
            ChartLab_vectorized_df.loc[ChartLab_vectorized_df.CHARTTIME>=T_Extub,off_vent_interventions] =\
            ChartLab_vectorized_df.loc[ChartLab_vectorized_df.CHARTTIME>=T_Extub,off_vent_interventions].ffill()
        # If never extubated:
        else:
            # ffil on_vent_interventions after intubation:
            ChartLab_vectorized_df.loc[(ChartLab_vectorized_df.CHARTTIME>=T_Intub),on_vent_interventions] =\
            ChartLab_vectorized_df.loc[(ChartLab_vectorized_df.CHARTTIME>=T_Intub),on_vent_interventions].ffill()
    
    # If not intubated in hostpital:
    else:
        # If an extubation seen:
        if pd.notnull(T_Extub):
        # ffil on_vent_interventions before extubation:
            ChartLab_vectorized_df.loc[(ChartLab_vectorized_df.CHARTTIME<T_Extub),on_vent_interventions] =\
            ChartLab_vectorized_df.loc[(ChartLab_vectorized_df.CHARTTIME<T_Extub),on_vent_interventions].ffill()
        
        # If not extubated either:
        else:
            # ffil off_vent_interventions all the way
            ChartLab_vectorized_df.loc[:,off_vent_interventions] = ChartLab_vectorized_df.loc[:,off_vent_interventions].ffill()
    return ChartLab_vectorized_df

In [42]:
def weighted_mean_measure_finder(df, Time_column_name = 'hours_since_T_i', measure = 'FiO2', T_range = [0,4]):
    '''
    This fucntion finds the weighted mean of measure (e.g., 'PEEP') within time_windows (in days) after T0. 
    Weight of each value is duration of the value. 
    The function cuts off anything outside of the range. 
    '''    
    if len(df)>0:
        cumulative_T = df[Time_column_name].tolist()[0]
        weighted_sum = cumulative_T*df[measure].tolist()[0]
        for index, row in df.iterrows():
            cumulative_T     += row['Length']
            if cumulative_T  < (T_range[1]-T_range[0]):
                weighted_sum += row['Length']*row[measure]                
            else:
                weighted_sum += (T_range[1]-row[Time_column_name])*row[measure]
        return round(weighted_sum/(T_range[1]-T_range[0]),2)  # Rounding it because there are a lot of E-14, which are zero, but are counted as non-zero and messing up the reuslts
    else:
        return np.NaN

In [43]:
def time_series_finder(hadm_ID, t_intubs_dict, conn=connection):
    import warnings
    import datetime

    warnings.filterwarnings("ignore", 'This pattern has match groups')
    
    
    time_series_df = chartlab_vectorizer(hadm_ID, connection)
    T_intub      = min([T for T in t_intubs_dict[hadm_ID] if (isinstance(T, datetime.datetime) & pd.notnull(T))])
#     print(T_intub)
    T_v = t_intubs_dict[hadm_ID][0]
    T_extub = T_extub_finder(hadm_ID,time_series_df, connection)
    
    if len(time_series_df)>0:
        time_series_df['hours_since_T_i'] = (time_series_df['CHARTTIME'] - T_intub).dt.total_seconds()/3600
        time_series_df['hours_since_T_v'] = (time_series_df['CHARTTIME'] - T_v).dt.total_seconds()/3600
    else:
        time_series_df['hours_since_T_i'] = np.NaN
        time_series_df['hours_since_T_v'] = np.NaN
    
    time_series_df = fill_forward_interventions (time_series_df, T_intub, T_extub)
    time_series_df['Length'] = -time_series_df['hours_since_T_i'].diff(-1)
    print(T_intub)
    print(T_extub)
    return {hadm_ID: time_series_df}

In [44]:
time_series_finder(102912, T_intubs_dict_for_analysis)[102912]

2166-09-24 12:16:00
2166-09-24 16:00:00


,SUBJECT_ID,HADM_ID,CHARTTIME,FiO2,Mean_Airway_P,O2_Device,O2_Device2,O2_Flow,O2_Flow2,PEEP,PH,PaCO2,PaO2,Peak,Plateau_P,RR,RR_Set,RR_Spont,RR_Total,SaO2,SpO2,TCO2,TV_Obsed,TV_Set,TV_Spont,Vent_Mode,Vent_Type,hours_since_T_i,hours_since_T_v,Length
0,70273.0,102912.0,2166-09-22 19:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drager,-41.266667,-41.266667,0.350000
1,70273.0,102912.0,2166-09-22 19:21:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN,Drager,-40.916667,-40.916667,0.100000
2,70273.0,102912.0,2166-09-22 19:27:00,40.0,8,Tracheostomy tube,NaN,NaN,NaN,5,NaN,NaN,NaN,16,NaN,NaN,NaN,29,NaN,NaN,NaN,NaN,472,NaN,425,CPAP/PSV,Drager,-40.816667,-40.816667,0.550000
3,70273.0,102912.0,2166-09-22 20:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,97,NaN,NaN,NaN,NaN,CPAP/PSV,Drager,-40.266667,-40.266667,1.000000
4,70273.0,102912.0,2166-09-22 21:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,98,NaN,NaN,NaN,NaN,CPAP/PSV,Drager,-39.266667,-39.266667,1.000000
5,70273.0,102912.0,2166-09-22 22:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,97,NaN,NaN,NaN,NaN,CPAP/PSV,Drager,-38.266667,-38.266667,1.000000
6,70273.0,102912.0,2166-09-22 23:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,CPAP/PSV,Drager,-37.266667,-37.266667,1.000000
7,70273.0,102912.0,2166-09-23 00:00:00,40.0,7,Tracheostomy tube,NaN,NaN,NaN,5,NaN,NaN,NaN,15,NaN,15,NaN,14,14,NaN,96,NaN,455,NaN,410,CPAP/PSV,Drager,-36.266667,-36.266667,1.000000
8,70273.0,102912.0,2166-09-23 01:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,NaN,93,NaN,NaN,NaN,NaN,CPAP/PSV,Drager,-35.266667,-35.266667,1.000000
9,70273.0,102912.0,2166-09-23 02:00:00,NaN,NaN,Tracheostomy tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,95,NaN,NaN,NaN,NaN,CPAP/PSV,Drager,-34.266667,-34.266667,1.000000


## **Finding the time series for all (CareVue):**

In [ ]:
start_time = time.time()
counter    = 0

time_series_dict_CareVue = dict()
for hadmID in adult_nonSurg_intubated_carevue_hadm_IDs:
    hadm_time_series = time_series_finder(hadmID, T_intubs_dict_for_analysis, conn=connection)
    time_series_dict_CareVue.update(hadm_time_series)
    counter+=1
    if counter%500 == 0:
        print('-----{} of {} done in {:.2f} minutes!'.format(counter,len(adult_nonSurg_intubated_carevue_hadm_IDs),(time.time() - start_time)/60))
        with open('time_series_dict_CareVue','wb') as file:
            pickle.dump(time_series_dict_CareVue, file)
            
with open('time_series_dict_CareVue','wb') as file:
    pickle.dump(time_series_dict_CareVue, file)

## **Finding the time series for all (MetaVision):**

In [38]:
start_time = time.time()
counter    = 0

# time_series_dict_MetaVision = dict()
for hadmID in adult_nonSurg_intubated_metavision_hadm_IDs:
    if hadmID in time_series_dict_MetaVision.keys():
        pass
    else:
        hadm_time_series = time_series_finder(hadmID, T_intubs_dict_for_analysis, conn=connection)
        time_series_dict_MetaVision.update(hadm_time_series)
        counter+=1
        if counter%500 == 0:
            print('-----{} of {} done in {:.2f} minutes!'.format(counter,len(adult_nonSurg_intubated_carevue_hadm_IDs),(time.time() - start_time)/60))
            with open('time_series_dict_MetaVision','wb') as file:
                pickle.dump(time_series_dict_MetaVision, file)
            
with open('time_series_dict_MetaVision','wb') as file:
    pickle.dump(time_series_dict_MetaVision, file)

In [39]:
len(adult_nonSurg_intubated_metavision_hadm_IDs)

3600

_____
# **Effect of $T_{intub}$ on clinical variables:**

## **Effect of $T_{intub}$ on mean of different clinical measures within `T_range` of intubation:**

In [167]:
def t_intub_effect_analyzer(hadm_ID, T_intubs_dict, time_series_dict,
                            measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'],
                            T_range = [0,4], max_possible_PtoF = 700, max_possible_PEEP = 25,conn=connection):
    '''
    This functions finds the average of each measure during T_range (in hours) based on T_intub and T_vent_paramter.
    '''
    import warnings
    import datetime

    warnings.filterwarnings("ignore", 'This pattern has match groups')
    
    time_series_df = time_series_dict[hadm_ID]
    
    # To enhance efficiency mimic, I will keep only what we need:
    time_series_df = time_series_df[[column for column in ['CHARTTIME','hours_since_T_i','hours_since_T_v','Length']+measures+['Plateau_P','TV_Obsed'] if column not in ['PtoF', 'StoF','static_compliance']]].copy()
    
    
    
    # 'FiO2' set to the first value when we had for cases with delata_t>0. 
    try:
        time_series_df.loc[(time_series_df['hours_since_T_i']>=0)
                           &(time_series_df['hours_since_T_v']<0)
                           &(pd.isnull(time_series_df['FiO2'])),'FiO2'] = time_series_df[time_series_df['hours_since_T_v']>=0]['FiO2'].tolist()[0]
    except IndexError:
        pass
     
    time_series_df.loc[time_series_df['FiO2']>0,'PtoF']   = 100*time_series_df['PaO2'].astype('float64')/time_series_df['FiO2'].astype('float64')
    time_series_df.loc[time_series_df['FiO2']>0,'StoF']   = 100*time_series_df['SpO2'].astype('float64')/time_series_df['FiO2'].astype('float64')
    time_series_df.loc[time_series_df['Plateau_P']!=time_series_df['PEEP'],'static_compliance'] = time_series_df['TV_Obsed'].astype('float64')/(time_series_df['Plateau_P'].astype('float64')-time_series_df['PEEP'].astype('float64'))
    
    # Cleaning:
    time_series_df.loc[time_series_df['PEEP'] > max_possible_PEEP, 'PEEP'] = np.NaN
    time_series_df.loc[time_series_df['PtoF'] > max_possible_PtoF, 'PtoF'] = np.NaN 
#    
    
    time_series_within_T_i_range_df = time_series_df[(time_series_df['hours_since_T_i']>=T_range[0]) & (time_series_df['hours_since_T_i']<=T_range[1])].copy()
    time_series_within_T_v_range_df = time_series_df[(time_series_df['hours_since_T_v']>=T_range[0]) & (time_series_df['hours_since_T_v']<=T_range[1])].copy()

    
    mean_within_T_i_range_dict = dict()
    mean_within_T_v_range_dict = dict()
    for measure in measures:
        if measure in ['PEEP','FiO2']:
            weighted_mean_T_i = weighted_mean_measure_finder(time_series_within_T_i_range_df.dropna(subset=[measure]), Time_column_name = 'hours_since_T_i', measure = measure, T_range = T_range)
            mean_within_T_i_range_dict.update({measure:weighted_mean_T_i})
            
            weighted_mean_T_v = weighted_mean_measure_finder(time_series_within_T_v_range_df.dropna(subset=[measure]), Time_column_name = 'hours_since_T_v', measure = measure, T_range = T_range)
            mean_within_T_v_range_dict.update({measure:weighted_mean_T_v})
        else:
            mean_within_T_i_range = time_series_within_T_i_range_df[measure].mean()
            mean_within_T_i_range_dict.update({measure:mean_within_T_i_range})
            
            mean_within_T_v_range = time_series_within_T_v_range_df[measure].mean()
            mean_within_T_v_range_dict.update({measure:mean_within_T_v_range})
    
    mean_within_T_i_range = pd.Series(mean_within_T_i_range_dict) 
    mean_within_T_v_range = pd.Series(mean_within_T_v_range_dict)
        
    return {hadm_ID: (mean_within_T_i_range,mean_within_T_v_range)}

In [187]:
test_hadmID = random.sample(set(adult_nonSurg_hadm_IDs)-set(not_intubated_hadmIDs), k=1)[0]
test_time_series_dict = time_series_finder(test_hadmID, T_intubs_dict_for_analysis, conn=connection)
t_intub_effect_analyzer(test_hadmID, T_intubs_dict_for_analysis, test_time_series_dict,
                        measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'], T_range = [0,12],
                        max_possible_PtoF = 700, max_possible_PEEP = 25,conn=connection)[test_hadmID]

2199-08-03 20:44:00
2199-08-20 07:28:00


(FiO2                  66.720000
 PEEP                   5.000000
 SpO2                  99.916667
 PaO2                 193.000000
 PH                     7.370000
 PaCO2                 37.000000
 PtoF                 241.000000
 StoF                 166.500000
 static_compliance     47.600000
 dtype: float64,
 FiO2                  66.720000
 PEEP                   5.000000
 SpO2                  99.916667
 PaO2                 193.000000
 PH                     7.370000
 PaCO2                 37.000000
 PtoF                 241.000000
 StoF                 166.500000
 static_compliance     47.600000
 dtype: float64)

In [169]:
test_hadmID

102912

In [42]:
def T_intub_effect_summarizer(ID_list, tintub_dict, time_series_dict,
                              measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'], 
                              T_range= [0,4]):
    '''
    This function summarizes the effects.
    '''
  
    measures_means_dict = dict()
    start_time = time.time()
    counter    = 0
    for ID in ID_list:
        result = t_intub_effect_analyzer(ID, tintub_dict, time_series_dict, measures,T_range)
        measures_means_dict.update(result)
        counter+=1
        if counter%500 == 0:
            print('-----{} of {} done in {:.2f} minutes!'.format(counter,len(ID_list),(time.time() - start_time)/60))
    print('Measures_means_dict completed!')

            
            
    # Printing the results -------------------------------------------------------------------------------------
    table_dict = dict()
    for measure_name in measures:
        mean_T_i = []
        mean_T_v = []
        delta_mean_list = []

        for ID in ID_list:
            ID_T_i_measure = measures_means_dict[ID][0][measure_name]
            ID_T_v_measure = measures_means_dict[ID][1][measure_name]
            mean_T_i.append(ID_T_i_measure)
            mean_T_v.append(ID_T_v_measure)
            delta_mean_list.append(ID_T_i_measure-ID_T_v_measure)

        non_nan_abs_delta_list  = [abs(i) for i in delta_mean_list if ~np.isnan(i)]
        non_zero_abs_delta_list = [abs(i) for i in delta_mean_list if (~np.isnan(i)) and (i!=0)]
        non_zero_delta_list     = [i for i in delta_mean_list if (~np.isnan(i)) and (i!=0)]

        text = ('''{:,} ({:.1f}%)
{:.2f} ({:.2f})
{:.2f} ({:.2f}-{:.2f})'''.format(len(non_zero_abs_delta_list),          
                len(non_zero_abs_delta_list)/len(non_nan_abs_delta_list)*100,
                np.mean(non_zero_abs_delta_list), 
                np.std(non_zero_abs_delta_list,ddof=1),
                np.median(non_zero_abs_delta_list),
                np.percentile(non_zero_abs_delta_list,25),
                np.percentile(non_zero_abs_delta_list,75)
               ))
        table_dict.update({measure_name:text})
        
    return (measures_means_dict, pd.Series(table_dict))

## **CareVue**: 

In [43]:
with open('time_series_dict_CareVue','rb') as file:
    time_series_dict_CareVue = pickle.load(file)

In [59]:
# measures_means_dict_given_T_range_CareVue = dict()
for t_range in [2,4,8,12]:
    means_dict, table_results = T_intub_effect_summarizer(adult_nonSurg_intubated_carevue_hadm_IDs,T_intubs_dict_for_analysis,time_series_dict_CareVue,
                                                          measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'], T_range= [0,t_range])
    measures_means_dict_given_T_range_CareVue.update({t_range: means_dict})
    table_results.to_excel('impact_table_carevue_{}_hours.xlsx'.format(t_range))

-----500 of 4657 done in 0.37 minutes!
-----1000 of 4657 done in 0.75 minutes!
-----1500 of 4657 done in 1.14 minutes!
-----2000 of 4657 done in 1.52 minutes!
-----2500 of 4657 done in 1.90 minutes!
-----3000 of 4657 done in 2.28 minutes!
-----3500 of 4657 done in 2.67 minutes!
-----4000 of 4657 done in 3.05 minutes!
-----4500 of 4657 done in 3.43 minutes!
Measures_means_dict completed!


## **MetaVision:**

In [61]:
with open('time_series_dict_MetaVision','rb') as file:
    time_series_dict_MetaVision = pickle.load(file)

In [62]:
measures_means_dict_given_T_range_MetaVision = dict()
for t_range in [2,4,8,12]:
    means_dict, table_results = T_intub_effect_summarizer(adult_nonSurg_intubated_metavision_hadm_IDs,T_intubs_dict_for_analysis,time_series_dict_MetaVision,
                                                          measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'], T_range= [0,t_range])
    measures_means_dict_given_T_range_MetaVision.update({t_range: means_dict})
    table_results.to_excel('impact_table_metavision_{}_hours.xlsx'.format(t_range))

-----500 of 3600 done in 0.33 minutes!
-----1000 of 3600 done in 0.66 minutes!
-----1500 of 3600 done in 1.00 minutes!
-----2000 of 3600 done in 1.33 minutes!
-----2500 of 3600 done in 1.66 minutes!
-----3000 of 3600 done in 1.99 minutes!
-----3500 of 3600 done in 2.33 minutes!
Measures_means_dict completed!
-----500 of 3600 done in 0.34 minutes!
-----1000 of 3600 done in 0.67 minutes!
-----1500 of 3600 done in 1.01 minutes!
-----2000 of 3600 done in 1.37 minutes!
-----2500 of 3600 done in 1.71 minutes!
-----3000 of 3600 done in 2.05 minutes!
-----3500 of 3600 done in 2.39 minutes!
Measures_means_dict completed!
-----500 of 3600 done in 0.35 minutes!
-----1000 of 3600 done in 0.69 minutes!
-----1500 of 3600 done in 1.04 minutes!
-----2000 of 3600 done in 1.39 minutes!
-----2500 of 3600 done in 1.74 minutes!
-----3000 of 3600 done in 2.09 minutes!
-----3500 of 3600 done in 2.44 minutes!
Measures_means_dict completed!
-----500 of 3600 done in 0.37 minutes!
-----1000 of 3600 done in 0.73 

____
## **Effect of $T_{intub}$ on the first recorded values of different clinical measures within `T_rang` of intubation:**

In [41]:
def t_intub_on_first_measurement_effect_analyzer(hadm_ID, T_intubs_dict, time_series_dict,
                            measures = ['FiO2',  'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'],
                            T_range = [0,4], max_possible_PtoF = 700, max_possible_PEEP = 25,conn=connection):
    # This functions finds the average of each measure during T_range (in hours) based on T_intub and T_vent_paramter.
    import warnings
    import datetime

    warnings.filterwarnings("ignore", 'This pattern has match groups')
    
    time_series_df = time_series_dict[hadm_ID]
    
    # To enhance efficiency mimic, I will keep only what we need:
    time_series_df = time_series_df[[column for column in ['CHARTTIME','hours_since_T_i','hours_since_T_v','Length']+measures+['Plateau_P','TV_Obsed'] if column not in ['PtoF', 'StoF','static_compliance']]].copy()
    
    
    
    # 'FiO2' set to the first value when we had for cases with delata_t>0. 
    try:
        time_series_df.loc[(time_series_df['hours_since_T_i']>=0)
                           &(time_series_df['hours_since_T_v']<0)
                           &(pd.isnull(time_series_df['FiO2'])),'FiO2'] = time_series_df[time_series_df['hours_since_T_v']>=0]['FiO2'].tolist()[0]
    except IndexError:
        pass
     
    time_series_df.loc[time_series_df['FiO2']>0,'PtoF']   = 100*time_series_df['PaO2'].astype('float64')/time_series_df['FiO2'].astype('float64')
    time_series_df.loc[time_series_df['FiO2']>0,'StoF']   = 100*time_series_df['SpO2'].astype('float64')/time_series_df['FiO2'].astype('float64')
    time_series_df.loc[time_series_df['Plateau_P']!=time_series_df['PEEP'],'static_compliance'] = time_series_df['TV_Obsed'].astype('float64')/(time_series_df['Plateau_P'].astype('float64')-time_series_df['PEEP'].astype('float64'))
    
    # Cleaning:
    time_series_df.loc[time_series_df['PEEP'] > max_possible_PEEP, 'PEEP'] = np.NaN
    time_series_df.loc[time_series_df['PtoF'] > max_possible_PtoF, 'PtoF'] = np.NaN 
#    
    
    time_series_within_T_i_range_df = time_series_df[(time_series_df['hours_since_T_i']>=T_range[0]) & (time_series_df['hours_since_T_i']<=T_range[1])].copy()
    time_series_within_T_v_range_df = time_series_df[(time_series_df['hours_since_T_v']>=T_range[0]) & (time_series_df['hours_since_T_v']<=T_range[1])].copy()
#     display(time_series_within_T_i_range_df)
#     display(time_series_within_T_v_range_df)
    
    first_within_T_i_range_dict = dict()
    first_within_T_v_range_dict = dict()
    for measure in measures:
        try:
            first_within_T_i_range = time_series_within_T_i_range_df[pd.notnull(time_series_within_T_i_range_df[measure])][measure].tolist()[0]
        except IndexError:
            first_within_T_i_range = np.NaN
        first_within_T_i_range_dict.update({measure:first_within_T_i_range})
        
        try:
            first_within_T_v_range = time_series_within_T_v_range_df[pd.notnull(time_series_within_T_v_range_df[measure])][measure].tolist()[0]
        
        except IndexError:
            first_within_T_v_range = np.NaN
        first_within_T_v_range_dict.update({measure:first_within_T_v_range})
    
    first_within_T_i_range = pd.Series(first_within_T_i_range_dict) 
    first_within_T_v_range = pd.Series(first_within_T_v_range_dict)
       
    return {hadm_ID: (first_within_T_i_range,first_within_T_v_range)}

def T_intub_effect_on_first_summarizer(ID_list, tintub_dict, time_series_dict, measures = ['FiO2', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF'], T_range= [0,4]):
    '''
    
    '''
    measures_means_dict = dict()
    start_time = time.time()
    counter    = 0
    for ID in ID_list:
        result = t_intub_on_first_measurement_effect_analyzer(ID, tintub_dict, time_series_dict, measures,T_range)
        measures_means_dict.update(result)
        counter+=1
        if counter%500 == 0:
            print('-----{} of {} done in {:.2f} minutes!'.format(counter,len(ID_list),(time.time() - start_time)/60))
    print('Measures_means_dict completed!')
        
    # Printing the results -------------------------------------------------------------------------------------
    table_dict = dict()
    for measure_name in measures:
        mean_T_i = []
        mean_T_v = []
        delta_mean_list = []

        for ID in ID_list:
            ID_T_i_measure = measures_means_dict[ID][0][measure_name]
            ID_T_v_measure = measures_means_dict[ID][1][measure_name]
            mean_T_i.append(ID_T_i_measure)
            mean_T_v.append(ID_T_v_measure)
            delta_mean_list.append(ID_T_i_measure-ID_T_v_measure)
    #         if ID_T_i_measure-ID_T_v_measure>100:
    #             print(ID)

        non_nan_abs_delta_list  = [abs(i) for i in delta_mean_list if ~np.isnan(i)]
        non_zero_abs_delta_list = [abs(i) for i in delta_mean_list if (~np.isnan(i)) and (i!=0)]
        non_zero_delta_list     = [i for i in delta_mean_list if (~np.isnan(i)) and (i!=0)]
        
        try:
            print('====================')
            print(measure_name)
            print('Count of missing x-hour mean {} based on T_i: {:,}'.format(measure_name,len([i for i in mean_T_i if np.isnan(i)])))
            print('Count of missing x-hour mean {} based on T_v: {:,}'.format(measure_name,len([i for i in mean_T_v if np.isnan(i)])))

            print('''
    Percent positive: {:.2f}%
    Percent negative: {:.2f}%'''.format(
            100*len([i for i in non_zero_delta_list if i>0])/len(non_zero_delta_list),
            100*len([i for i in non_zero_delta_list if i<0])/len(non_zero_delta_list)
             ))
            text = ('''{:,} ({:.1f}%)
    {:.2f} ({:.2f})
    {:.2f} ({:.2f}-{:.2f})'''.format(len(non_zero_abs_delta_list),          
                    len(non_zero_abs_delta_list)/len(non_nan_abs_delta_list)*100,
                    np.mean(non_zero_abs_delta_list), 
                    np.std(non_zero_abs_delta_list,ddof=1),
                    np.median(non_zero_abs_delta_list),
                    np.percentile(non_zero_abs_delta_list,25),
                    np.percentile(non_zero_abs_delta_list,75)
                   ))
        except ZeroDivisionError:
            pass
        
        table_dict.update({measure_name:text})
#         print(text)
        
    return (measures_means_dict, pd.Series(table_dict))

## **CareVue**: 

In [42]:
with open('time_series_dict_CareVue','rb') as file:
    time_series_dict_CareVue = pickle.load(file)

In [43]:
measures_firsts_dict_given_T_range_CareVue = dict()

for t_range in [4]:
    firsts_dict, firsts_table_results = T_intub_effect_on_first_summarizer(adult_nonSurg_intubated_carevue_hadm_IDs,T_intubs_dict_for_analysis,
                                                                           time_series_dict_CareVue,
                                                                           measures = ['FiO2', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','PEEP'], T_range= [0,t_range])
    measures_firsts_dict_given_T_range_CareVue.update({t_range: firsts_dict})
    firsts_table_results.to_excel('impact_table_carevue_first_{}_hours.xlsx'.format(t_range))

-----500 of 4657 done in 0.40 minutes!
-----1000 of 4657 done in 0.81 minutes!
-----1500 of 4657 done in 1.22 minutes!
-----2000 of 4657 done in 1.61 minutes!
-----2500 of 4657 done in 1.99 minutes!
-----3000 of 4657 done in 2.39 minutes!
-----3500 of 4657 done in 2.80 minutes!
-----4000 of 4657 done in 3.22 minutes!
-----4500 of 4657 done in 3.62 minutes!
Measures_means_dict completed!
FiO2
Count of missing x-hour mean FiO2 based on T_i: 103
Count of missing x-hour mean FiO2 based on T_v: 127

    Percent positive: 77.78%
    Percent negative: 22.22%
SpO2
Count of missing x-hour mean SpO2 based on T_i: 78
Count of missing x-hour mean SpO2 based on T_v: 120

    Percent positive: 45.11%
    Percent negative: 54.89%
PaO2
Count of missing x-hour mean PaO2 based on T_i: 1,404
Count of missing x-hour mean PaO2 based on T_v: 1,414

    Percent positive: 48.21%
    Percent negative: 51.79%
PH
Count of missing x-hour mean PH based on T_i: 1,376
Count of missing x-hour mean PH based on T_v: 1,

## **MetaVision**: 

In [44]:
with open('time_series_dict_MetaVision','rb') as file:
    time_series_dict_MetaVision = pickle.load(file)

In [45]:
measures_firsts_dict_given_T_range_MetaVision = dict()

for t_range in [4]:
    firsts_dict, firsts_table_results = T_intub_effect_on_first_summarizer(adult_nonSurg_intubated_metavision_hadm_IDs,T_intubs_dict_for_analysis,
                                                                           time_series_dict_MetaVision,
                                                                           measures = ['FiO2', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','PEEP'], T_range= [0,t_range])
    measures_firsts_dict_given_T_range_MetaVision.update({t_range: firsts_dict})
    firsts_table_results.to_excel('impact_table_metavision_first_{}_hours.xlsx'.format(t_range))

-----500 of 3600 done in 0.39 minutes!
-----1000 of 3600 done in 0.78 minutes!
-----1500 of 3600 done in 1.17 minutes!
-----2000 of 3600 done in 1.54 minutes!
-----2500 of 3600 done in 1.91 minutes!
-----3000 of 3600 done in 2.29 minutes!
-----3500 of 3600 done in 2.66 minutes!
Measures_means_dict completed!
FiO2
Count of missing x-hour mean FiO2 based on T_i: 92
Count of missing x-hour mean FiO2 based on T_v: 169

    Percent positive: 77.11%
    Percent negative: 22.89%
SpO2
Count of missing x-hour mean SpO2 based on T_i: 42
Count of missing x-hour mean SpO2 based on T_v: 138

    Percent positive: 45.51%
    Percent negative: 54.49%
PaO2
Count of missing x-hour mean PaO2 based on T_i: 1,392
Count of missing x-hour mean PaO2 based on T_v: 1,449

    Percent positive: 50.51%
    Percent negative: 49.49%
PH
Count of missing x-hour mean PH based on T_i: 1,362
Count of missing x-hour mean PH based on T_v: 1,419

    Percent positive: 32.99%
    Percent negative: 67.01%
PaCO2
Count of mis

___
# **Effect of $T_i$ time window on missingness and measure means:**

In [52]:
def t_effect_on_missing_and_mean_summarizer(measures_means_dict_given_t, measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'],t_list = [2,4,8,12,24]):
    
    missingness_mean_table = pd.DataFrame(columns = ['missingness', 'mean'])
    for measure_name in measures:        
        missing_string = ''
        mean_string = ''
        for t in t_list:
            measures_means_dict = measures_means_dict_given_t[t]
            ID_list = measures_means_dict.keys()
            mean_T_i = []
            mean_T_v = []
            delta_mean_list = []
            for ID in ID_list:
                ID_T_i_measure = measures_means_dict[ID][0][measure_name]
                ID_T_v_measure = measures_means_dict[ID][1][measure_name]
                mean_T_i.append(ID_T_i_measure)
                mean_T_v.append(ID_T_v_measure)
                delta_mean_list.append(ID_T_i_measure-ID_T_v_measure)
                
            non_nan_abs_delta_list  = [abs(i) for i in delta_mean_list if ~np.isnan(i)]
            non_zero_abs_delta_list = [abs(i) for i in delta_mean_list if (~np.isnan(i)) and (i!=0)]

            n_missing       = len(ID_list)-len(non_nan_abs_delta_list)
            percent_missing = n_missing/len(ID_list)*100
            mean = np.mean(non_zero_abs_delta_list)
            missing_string += '{:3,} ({:4.1f}%)\n'.format(n_missing, percent_missing)
            mean_string    += '{:.2f}\n'.format(mean)
        
        missingness_mean_table.loc[measure_name, 'missingness'] = missing_string
        missingness_mean_table.loc[measure_name, 'mean'] = mean_string
    return missingness_mean_table

## **CareVue**: 

In [54]:
missingness_mean_table_given_t = t_effect_on_missing_and_mean_summarizer(measures_means_dict_given_T_range_CareVue, measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'],t_list = [2,4,8,12])
missingness_mean_table_given_t.to_excel('missingness_mean_table_given_t_carevue.xlsx')
missingness_mean_table_given_t

,missingness,mean
FiO2,200 ( 4.3%)\n132 ( 2.8%)\n106 ( 2.3%)\n 93 ( 2.0%)\n,17.55\n10.50\n5.68\n3.99\n
PEEP,147 ( 3.2%)\n 98 ( 2.1%)\n 81 ( 1.7%)\n 80 ( 1.7%)\n,1.62\n1.04\n0.61\n0.41\n
SpO2,149 ( 3.2%)\n120 ( 2.6%)\n 95 ( 2.0%)\n 81 ( 1.7%)\n,1.11\n0.71\n0.53\n0.53\n
PaO2,"2,442 (52.4%)\n1,430 (30.7%)\n875 (18.8%)\n721 (15.5%)\n",82.35\n49.50\n31.74\n25.52\n
PH,"2,417 (51.9%)\n1,402 (30.1%)\n855 (18.4%)\n705 (15.1%)\n",0.07\n0.05\n0.04\n0.03\n
PaCO2,"2,442 (52.4%)\n1,430 (30.7%)\n875 (18.8%)\n721 (15.5%)\n",8.85\n6.87\n4.96\n4.42\n
PtoF,"3,599 (77.3%)\n2,944 (63.2%)\n2,380 (51.1%)\n2,077 (44.6%)\n",66.44\n53.34\n31.50\n28.28\n
StoF,"1,060 (22.8%)\n539 (11.6%)\n296 ( 6.4%)\n250 ( 5.4%)\n",26.53\n20.48\n17.14\n14.46\n
static_compliance,"1,376 (29.5%)\n1,083 (23.3%)\n819 (17.6%)\n722 (15.5%)\n",9.52\n8.94\n3.26\n4.54\n


## **MetaVision**: 

In [66]:
missingness_mean_table_given_t = t_effect_on_missing_and_mean_summarizer(measures_means_dict_given_T_range_MetaVision, measures = ['FiO2', 'PEEP', 'SpO2', 'PaO2', 'PH', 'PaCO2', 'PtoF', 'StoF','static_compliance'],t_list = [2,4,8,12])
missingness_mean_table_given_t.to_excel('missingness_mean_table_given_t_metavision.xlsx')

____
# **Effect  of $T_i$  on oxygenation level:**

In [55]:
def t_intub_effect_on_hypoxia_summarizer(ID_list, measures_means_dict):
    '''
    In the output, rows are T_i and columns are T_v.
    '''
    
    from collections import Counter
    hypoxia_change_from_Ti_to_Tv  = []
    
    for ID in ID_list:
        ID_PtoF_Ti = measures_means_dict[ID][0]['PtoF']
        ID_PtoF_Tv = measures_means_dict[ID][1]['PtoF']
        
        if pd.notnull(ID_PtoF_Ti) & pd.notnull(ID_PtoF_Tv):
            if ID_PtoF_Ti<=100:
                hypoxia_Ti = 'Severe'
            elif ID_PtoF_Ti<=200:
                hypoxia_Ti = 'Moderate'
            elif ID_PtoF_Ti<=300:
                hypoxia_Ti = 'Mild'
            else:
                hypoxia_Ti = 'Normal'
        
            if ID_PtoF_Tv<=100:
                hypoxia_Tv = 'Severe'
            elif ID_PtoF_Tv<=200:
                hypoxia_Tv = 'Moderate'
            elif ID_PtoF_Tv<=300:
                hypoxia_Tv = 'Mild'
            else:
                hypoxia_Tv = 'Normal'
        
            ID_hypoxia_change_from_Ti_to_Tv = hypoxia_Ti + '-' + hypoxia_Tv
            hypoxia_change_from_Ti_to_Tv.append(ID_hypoxia_change_from_Ti_to_Tv)
        else:
            pass
    total_count = sum(Counter(hypoxia_change_from_Ti_to_Tv).values())
    hypoxia_level_matrix = pd.DataFrame(columns = ['Normal','Mild','Moderate','Severe'])
    for item, count in Counter(hypoxia_change_from_Ti_to_Tv).items():
#         print('{}: {:,}({:.1f}%)'.format(item, count, count/total_count*100))
        hypoxia_level_matrix.loc[item.split('-')[0],item.split('-')[1]] =  '{:,} ({:.1f}%)'.format(count, count/total_count*100)
    hypoxia_level_matrix= hypoxia_level_matrix.reindex(['Normal','Mild','Moderate','Severe'])
    changed_hypoxia_level = sum([count for item, count in Counter(hypoxia_change_from_Ti_to_Tv).items() if item not in ['Mild-Mild','Moderate-Moderate','Severe-Severe','Normal-Normal', ]])
    print('n(%) non-missing hypoxia pairs:  {:,} ({:.1f}%)'.format(len(hypoxia_change_from_Ti_to_Tv),100*len(hypoxia_change_from_Ti_to_Tv)/len(ID_list)))
    print('n(%) with changed hypoxia level: {:,} ({:.1f}%)'.format(changed_hypoxia_level,changed_hypoxia_level/total_count*100))
    return hypoxia_level_matrix

## **CareVue:**

In [58]:
with pd.ExcelWriter('hypoxia_matrices_carevue.xlsx') as writer:
    for t_range  in [2,4,8,12]:
        print('\nWithin {} hours of intubation: '.format(t_range))
        hypoxia_matrix = t_intub_effect_on_hypoxia_summarizer(adult_nonSurg_intubated_carevue_hadm_IDs,measures_means_dict_given_T_range_CareVue[t_range])
        display(hypoxia_matrix)
        hypoxia_matrix.to_excel(writer,'Within {} hours'.format(t_range))
    writer.save()


Within 2 hours of intubation: 
n(%) non-missing hypoxia pairs:  1,058 (22.7%)
n(%) with changed hypoxia level: 6 (0.6%)


,Normal,Mild,Moderate,Severe
Normal,337 (31.9%),1 (0.1%),NaN,NaN
Mild,NaN,248 (23.4%),1 (0.1%),NaN
Moderate,2 (0.2%),NaN,295 (27.9%),NaN
Severe,NaN,NaN,2 (0.2%),172 (16.3%)



Within 4 hours of intubation: 
n(%) non-missing hypoxia pairs:  1,713 (36.8%)
n(%) with changed hypoxia level: 8 (0.5%)


,Normal,Mild,Moderate,Severe
Normal,517 (30.2%),NaN,NaN,NaN
Mild,1 (0.1%),442 (25.8%),3 (0.2%),NaN
Moderate,1 (0.1%),NaN,512 (29.9%),NaN
Severe,NaN,NaN,3 (0.2%),234 (13.7%)



Within 8 hours of intubation: 
n(%) non-missing hypoxia pairs:  2,277 (48.9%)
n(%) with changed hypoxia level: 4 (0.2%)


,Normal,Mild,Moderate,Severe
Normal,671 (29.5%),NaN,NaN,NaN
Mild,NaN,632 (27.8%),NaN,NaN
Moderate,1 (0.0%),2 (0.1%),734 (32.2%),NaN
Severe,NaN,NaN,1 (0.0%),236 (10.4%)



Within 12 hours of intubation: 
n(%) non-missing hypoxia pairs:  2,580 (55.4%)
n(%) with changed hypoxia level: 11 (0.4%)


,Normal,Mild,Moderate,Severe
Normal,757 (29.3%),1 (0.0%),NaN,NaN
Mild,NaN,736 (28.5%),NaN,NaN
Moderate,1 (0.0%),6 (0.2%),842 (32.6%),1 (0.0%)
Severe,NaN,NaN,2 (0.1%),234 (9.1%)


## **MetaVision**: 

In [67]:
with pd.ExcelWriter('hypoxia_matrices_metavision.xlsx') as writer:
    for t_range  in [2,4,8,12]:
        print('\nWithin {} hours of intubation: '.format(t_range))
        hypoxia_matrix = t_intub_effect_on_hypoxia_summarizer(adult_nonSurg_intubated_metavision_hadm_IDs,measures_means_dict_given_T_range_MetaVision[t_range])
        display(hypoxia_matrix)
        hypoxia_matrix.to_excel(writer,'Within {} hours'.format(t_range))
    writer.save()


Within 2 hours of intubation: 
n(%) non-missing hypoxia pairs:  551 (15.3%)
n(%) with changed hypoxia level: 3 (0.5%)


,Normal,Mild,Moderate,Severe
Normal,149 (27.0%),1 (0.2%),NaN,NaN
Mild,NaN,129 (23.4%),NaN,NaN
Moderate,NaN,1 (0.2%),171 (31.0%),1 (0.2%)
Severe,NaN,NaN,NaN,99 (18.0%)



Within 4 hours of intubation: 
n(%) non-missing hypoxia pairs:  887 (24.6%)
n(%) with changed hypoxia level: 13 (1.5%)


,Normal,Mild,Moderate,Severe
Normal,245 (27.6%),1 (0.1%),NaN,NaN
Mild,1 (0.1%),213 (24.0%),3 (0.3%),NaN
Moderate,1 (0.1%),3 (0.3%),271 (30.6%),1 (0.1%)
Severe,NaN,NaN,3 (0.3%),145 (16.3%)



Within 8 hours of intubation: 
n(%) non-missing hypoxia pairs:  1,244 (34.6%)
n(%) with changed hypoxia level: 21 (1.7%)


,Normal,Mild,Moderate,Severe
Normal,323 (26.0%),4 (0.3%),1 (0.1%),NaN
Mild,3 (0.2%),319 (25.6%),4 (0.3%),NaN
Moderate,2 (0.2%),1 (0.1%),411 (33.0%),1 (0.1%)
Severe,NaN,NaN,5 (0.4%),170 (13.7%)



Within 12 hours of intubation: 
n(%) non-missing hypoxia pairs:  1,471 (40.9%)
n(%) with changed hypoxia level: 24 (1.6%)


,Normal,Mild,Moderate,Severe
Normal,393 (26.7%),4 (0.3%),NaN,NaN
Mild,2 (0.1%),395 (26.9%),5 (0.3%),NaN
Moderate,2 (0.1%),5 (0.3%),477 (32.4%),3 (0.2%)
Severe,NaN,NaN,3 (0.2%),182 (12.4%)
